In [10]:
import numpy as np
import pandas as pd
import torch
import os
os.environ ["KMP_DUPLICATE_LIB_OK"] ="TRUE"
from torch import nn
from d2l import torch as d2l
from torch.nn import functional as F

当需要更强的灵活性时，我们需要定义自己的块。 例如，我们可能希望在前向传播函数中执行Python的控制流。 此外，我们可能希望执行任意的数学运算， 而不是简单地依赖预定义的神经网络层。

In [11]:
# ()当中为该类继承的父类
class MLP(nn.Module):
    def __init__(self):
        super().__init__() # 父类的init函数
        self.hidden = nn.Linear(20,256)
        self.out = nn.Linear(256,10)

    def forward(self,X):
        return self.out(F.relu(self.hidden(X)))

In [14]:
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)


tensor([[0.5084, 0.0234, 0.2324, 0.8389, 0.2188, 0.0807, 0.2197, 0.9048, 0.5850,
         0.8006, 0.8420, 0.6595, 0.2179, 0.1998, 0.5880, 0.7368, 0.9461, 0.3790,
         0.8514, 0.0441],
        [0.7092, 0.6483, 0.0623, 0.7688, 0.0254, 0.7000, 0.8813, 0.0878, 0.5291,
         0.4179, 0.5552, 0.5370, 0.6381, 0.4795, 0.2424, 0.4252, 0.0023, 0.1759,
         0.8189, 0.2468]])


tensor([[ 0.2372, -0.0454, -0.2730,  0.0371,  0.0904,  0.0771,  0.0566,  0.1171,
         -0.1327, -0.1393],
        [ 0.2900, -0.0497, -0.2357,  0.1661,  0.0385,  0.0370, -0.1158,  0.0748,
         -0.1775, -0.0188]], grad_fn=<AddmmBackward0>)

In [12]:
net = MLP()
net(X)

tensor([[-0.0955,  0.2500,  0.1953, -0.0202,  0.0477,  0.0377, -0.1918, -0.1803,
          0.2415, -0.1874],
        [-0.0999,  0.2789,  0.2507, -0.0682, -0.0976,  0.0323, -0.1187, -0.1226,
          0.2054, -0.1948]], grad_fn=<AddmmBackward0>)

In [16]:
# 与Sequential类相同的功能
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()

        for idx,moudle in enumerate(args):
            # self._moudles是一个字典 
            self._modules[str(idx)] = moudle

    def forward(self,X):
        # 依次计算每个moudle的输出 并作为下一个moudle的输出
        for block in self._modules.values():
            X = block(X)
        
        return X   

In [17]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256,10))
net(X)


tensor([[-0.4050,  0.0859,  0.0217, -0.0808, -0.0752, -0.1994,  0.1723,  0.1233,
         -0.1319,  0.0438],
        [-0.1312,  0.0705,  0.0638, -0.1632, -0.0294, -0.2264,  0.0639,  0.0049,
         -0.0778,  0.0396]], grad_fn=<AddmmBackward0>)

In [ ]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20,20),requires_grad=False)
        self.linear = nn.Linear(20,20)

    
    def forward(self,X):
        X = self.linear(X)